In [ ]:
import pandas as pd

In [ ]:
df = pd.read_parquet("Production.parquet")
df[:5]

In [ ]:
df_wind = (
    df
    # .filter(regex="PriceArea|Wind")
    [["PriceArea","OnshoreWindGe50kW_MWh"]]
    # .assign(
    #     WIND_PRODUCTION=lambda df: df.sum(axis='columns')
    # )
    .reset_index()
    .query("PriceArea == 'DK1'")
    .query("HourUTC >= '2017-01-01'")
)

df_wind[:2]

In [ ]:
# df = (
#     df
#     .groupby(
#         [
#             "PriceArea",
#             pd.Grouper(freq="M"),
#         ],
#         # as_index=False
#     )
#     .agg(
#         PRODUCTION=("TOTAL_PRODUCTION", "sum")
#     )
#     .reset_index()
#     .set_index("HourUTC")
# )
# df

In [ ]:
# dk_1 = (
#     df
#     .query("PriceArea == 'DK1'")
#     .reset_index()
# )

# dk_2 = (
#     df
#     .query("PriceArea == 'DK2'")
#     .reset_index()
# )

In [ ]:
# dk_2[:5]

In [ ]:
df_wind_model = (
    df_wind
    .set_index("HourUTC") 
    .groupby(
        pd.Grouper(freq="D")
    )
    .agg(
        MEAN_PRODUCTION=("OnshoreWindGe50kW_MWh", "sum")
    )
    .rolling(30)
    .mean()
    # .plot(figsize=(30,10))
    .reset_index()
)

In [ ]:
from darts import TimeSeries

series = TimeSeries.from_dataframe(df_wind_model, "HourUTC", "MEAN_PRODUCTION")

In [ ]:
series.plot()

In [ ]:
NUM = 1


train, val = series[:-NUM], series[-NUM:]

In [ ]:
# from statistics import mode
# from darts.models import ExponentialSmoothing

# model = ExponentialSmoothing()
# model.fit(train)

# prediction = model.predict(len(val), num_samples=100)

In [ ]:
import matplotlib.pyplot as plt

# plt.figure( figsize=(30,10))
# series.plot()
# prediction.plot(label="forecast", low_quantile=0.2, high_quantile=0.8)
# plt.legend()

### !! Model saving does not work

See notebook `01_training_production_prophet.ipynb` where I import Prophet directly.

So instead of `from darts.models import Prophet` I use `from prophet import Prophet`.
This also means that instead of the `from darts import TimeSeries` API I use a regular `pd.DataFrame` as input for the model.


In [18]:
from darts.models import Prophet

In [ ]:
model = Prophet()
model.fit(train)


FUTURE = 10

prediction = model.predict(FUTURE, num_samples=100)

plt.figure(figsize=(30,10))
series[-60:].plot()
prediction[-60:].plot(label="forecast", low_quantile=0.1, high_quantile=0.9)
plt.legend();

In [ ]:
# Python
from prophet.serialize import model_to_json, model_from_json

with open('serialized_model.json', 'w') as fout:
    fout.write(model_to_json(model))  # Save model

with open('serialized_model.json', 'r') as fin:
    m = model_from_json(fin.read())  # Load model